<a href="https://colab.research.google.com/github/y-patankar/CPSC597/blob/main/CreditCard_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mlg-ulb/creditcardfraud
!unzip creditcardfraud.zip

 82% 54.0M/66.0M [00:00<00:00, 192MB/s]
100% 66.0M/66.0M [00:00<00:00, 173MB/s]
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [2]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
df= pd.read_csv('creditcard.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


Preprocessing the "Time" and "Amount" column

In [4]:
from sklearn.preprocessing import RobustScaler
new_df = df.copy()
new_df['Amount'] = RobustScaler().fit_transform(new_df['Amount'].to_numpy().reshape(-1, 1))
time = new_df['Time']
new_df['Time'] = (time - time.min()) / (time.max() - time.min())
new_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000000,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.783274,0
1,0.000000,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.269825,0
2,0.000006,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,0
3,0.000006,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,0
4,0.000012,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.670579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,0.999965,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.296653,0
284803,0.999971,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0.038986,0
284804,0.999977,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0.641096,0
284805,0.999977,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.167680,0


In [7]:
#shuffeling the rows
new_df = new_df.sample(frac=1, random_state=1)
new_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
41794,0.236284,-1.696681,1.462953,-1.185289,-1.697343,3.391338,2.807224,0.959570,0.122927,0.512893,...,-0.297149,-0.330778,-0.430592,0.900733,-0.043413,-0.055116,-1.633066,-1.140646,-0.296793,0
202234,0.776842,1.891791,-1.503085,-0.491290,-0.679490,-1.580169,-0.842999,-0.984041,-0.107069,0.297659,...,0.468191,1.032788,0.029336,0.058461,-0.295344,-0.079380,-0.017766,-0.022913,1.788584,0
244968,0.882877,-0.532351,1.245302,-0.525565,-0.126971,1.992338,-0.424851,1.440420,0.024171,-0.780995,...,0.043191,0.082510,-0.476516,0.124001,0.581086,-0.607337,0.090852,0.143801,-0.293440,0
127188,0.452781,-0.547412,1.034031,1.062279,-0.607164,1.029405,0.685552,0.706502,0.059287,-0.331181,...,-0.314018,-0.557295,-0.219467,-1.344487,-0.068709,0.141906,0.228209,-0.070068,-0.289387,0
677,0.002963,-0.656147,0.501881,1.532601,-1.319886,-0.186189,-0.897731,1.191646,-0.191943,-0.211756,...,-0.130362,-0.374358,0.112819,0.526267,-0.406443,0.597938,0.046437,-0.055101,0.810452,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21621,0.183857,1.214370,0.090817,0.677072,0.918274,-0.724679,-0.909902,-0.024860,-0.151567,0.459109,...,-0.213507,-0.431827,0.056888,0.740339,0.407431,0.249798,-0.022020,0.015413,-0.252498,0
237552,0.864010,-1.293565,0.306847,0.935595,-1.639104,-0.541232,-0.696428,-0.396424,0.530566,-1.288224,...,0.332182,0.704664,-0.306042,-0.044608,0.619868,0.021797,-0.184185,-0.011100,0.111088,0
200735,0.772935,0.839847,0.559181,-1.056850,-0.593976,1.118772,0.290124,0.277976,-0.922628,-0.132232,...,1.064203,0.713816,-0.252643,-0.291491,0.357535,-0.107797,0.286159,0.323320,-0.269825,0
250112,0.895493,0.018366,0.797151,0.175159,-0.773854,0.607998,-0.612039,0.914646,-0.040444,-0.281542,...,-0.227936,-0.519085,0.026252,-0.343708,-0.475729,0.140578,0.241616,0.084217,-0.212674,0


In [8]:
#split the train, test and validation class
train, test, val = new_df[:240000], new_df[240000:262000], new_df[262000:]
train['Class'].value_counts(), test['Class'].value_counts(), val['Class'].value_counts()


(0    239595
 1       405
 Name: Class, dtype: int64,
 0    21962
 1       38
 Name: Class, dtype: int64,
 0    22758
 1       49
 Name: Class, dtype: int64)

In [9]:
#panda dataframe to numpy df
train_np, test_np, val_np = train.to_numpy(), test.to_numpy(), val.to_numpy()
train_np.shape, test_np.shape, val_np.shape



((240000, 31), (22000, 31), (22807, 31))

In [10]:
#seperating the features and labels
x_train, y_train = train_np[:, :-1], train_np[:, -1]
x_test, y_test = test_np[:, :-1], test_np[:, -1]
x_val, y_val = val_np[:, :-1], val_np[:, -1]
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((240000, 30), (240000,), (22000, 30), (22000,), (22807, 30), (22807,))

Traning the model with unbalanced dataset.

In [11]:
# logistic regression object
lr = LogisticRegression()

# train the model on train set
lr.fit(x_train, y_train.ravel())

predictions = lr.predict(x_val)

# print classification report for validation
print(classification_report(y_val, predictions, target_names=['Not Fraud','Fraud']))
# print(classification_report(y_val,target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     22758
       Fraud       0.90      0.57      0.70        49

    accuracy                           1.00     22807
   macro avg       0.95      0.79      0.85     22807
weighted avg       1.00      1.00      1.00     22807



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Prediction using test data. (unbalanced dataset)

In [12]:
# print classification report for test data
lr = LogisticRegression()
lr.fit(x_train, y_train.ravel())

predictions = lr.predict(x_test)

print(classification_report(y_test, predictions, target_names=['Not Fraud','Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     21962
       Fraud       0.86      0.79      0.82        38

    accuracy                           1.00     22000
   macro avg       0.93      0.89      0.91     22000
weighted avg       1.00      1.00      1.00     22000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:

print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

Before OverSampling, counts of label '1': 405
Before OverSampling, counts of label '0': 239595 

After OverSampling, the shape of train_X: (479190, 30)
After OverSampling, the shape of train_y: (479190,) 

After OverSampling, counts of label '1': 239595
After OverSampling, counts of label '0': 239595


Traning the model using balanced dataset.

In [15]:
#classification after smote
lr_smote = LogisticRegression()
lr_smote.fit(X_train_res, y_train_res.ravel())
predictions = lr_smote.predict(x_test)

# print(classification_report(y_test, predictions))
print(classification_report(y_test, predictions, target_names=['Not Fraud','Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      0.97      0.99     21962
       Fraud       0.06      0.92      0.11        38

    accuracy                           0.97     22000
   macro avg       0.53      0.95      0.55     22000
weighted avg       1.00      0.97      0.99     22000



In [16]:
# Calculate confusion matrix after smote
cm = confusion_matrix(y_test, lr_smote.predict(x_test))

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[21386   576]
 [    3    35]]


In [17]:
# Calculate confusion matrix before smote
cm = confusion_matrix(y_test, lr.predict(x_test))

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[21957     5]
 [    8    30]]


In [18]:
import joblib


joblib.dump(lr_smote, 'lr_smote.pkl')

['lr_smote.pkl']

In [20]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import joblib

# Assuming 'model' is your trained model
# Save the model to a specific path on your Google Drive
joblib.dump(lr_smote, '/content/drive/MyDrive/CPSC-597/lr_smote.pkl')



['/content/drive/MyDrive/CPSC-597/lr_smote.pkl']

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

shallow_nn = Sequential()
shallow_nn.add(InputLayer((X_train_res.shape[1],)))
shallow_nn.add(Dense(2, 'relu'))
shallow_nn.add(BatchNormalization())
shallow_nn.add(Dense(1, 'sigmoid'))

checkpoint = ModelCheckpoint('shallow_nn', save_best_only=True)
shallow_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
shallow_nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 62        
                                                                 
 batch_normalization_2 (Bat  (None, 2)                 8         
 chNormalization)                                                
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 73 (292.00 Byte)
Trainable params: 69 (276.00 Byte)
Non-trainable params: 4 (16.00 Byte)
_________________________________________________________________


In [35]:
shallow_nn.fit(X_train_res, y_train_res, validation_data=(x_val, y_val), epochs=5, callbacks=checkpoint)

Epoch 1/5
14975/14975 [==============================] - 31s 2ms/step - loss: 0.2040 - accuracy: 0.9267 - val_loss: 0.0991 - val_accuracy: 0.9815
Epoch 2/5
14975/14975 [==============================] - 35s 2ms/step - loss: 0.1541 - accuracy: 0.9410 - val_loss: 0.0905 - val_accuracy: 0.9825
Epoch 3/5
14975/14975 [==============================] - 31s 2ms/step - loss: 0.1540 - accuracy: 0.9414 - val_loss: 0.0931 - val_accuracy: 0.9818
Epoch 4/5
14975/14975 [==============================] - 32s 2ms/step - loss: 0.1539 - accuracy: 0.9416 - val_loss: 0.0951 - val_accuracy: 0.9816
Epoch 5/5
14975/14975 [==============================] - 33s 2ms/step - loss: 0.1530 - accuracy: 0.9417 - val_loss: 0.0927 - val_accuracy: 0.9818


In [33]:
predictions = (shallow_nn.predict(x_test) > 0.5).astype("int32")

# Print classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, target_names=['Not Fraud', 'Fraud']))

688/688 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     21962
       Fraud       0.77      0.87      0.81        38

    accuracy                           1.00     22000
   macro avg       0.88      0.93      0.91     22000
weighted avg       1.00      1.00      1.00     22000



In [34]:
cm = confusion_matrix(y_test, predictions)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[21952    10]
 [    5    33]]


Untill this point, SMOTE is giving better accuracy in detecting fraud using linear regression(Fraud       0.06      0.92      0.11).
Sequencial neural network has slightly better precision and f1-score, lowering the accuracy of fetecting fraud. But model is significantly good. (0.77      0.87      0.81)




In [38]:
#implementing autoencoders

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, losses
import matplotlib.pyplot as plt

# Load the credit card transaction dataset
ae_data = pd.read_csv("creditcard.csv")

# Separate features and labels
ae_x = ae_data.drop("Class", axis=1)
ae_y = ae_data["Class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ae_x, ae_y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the autoencoder architecture
input_dim = X_train.shape[1]

model = tf.keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

model.compile(optimizer='adam', loss=losses.MeanSquaredError())

# Train the autoencoder on normal transactions
model.fit(X_train[y_train == 0], X_train[y_train == 0],
          epochs=10,
          batch_size=128,
          validation_data=(X_test, X_test))

# Reconstruct the test data and calculate reconstruction error
reconstructed = model.predict(X_test)
mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)
threshold = np.mean(mse) + 2 * np.std(mse)  # Set threshold as mean + 2*std deviation

# Predict fraud based on reconstruction error
y_pred = (mse > threshold).astype(int)

# Evaluate performance
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Epoch 1/10
1777/1777 [==============================] - 6s 3ms/step - loss: 0.7049 - val_loss: 0.6791
Epoch 2/10
1777/1777 [==============================] - 4s 2ms/step - loss: 0.6496 - val_loss: 0.6710
Epoch 3/10
1777/1777 [==============================] - 4s 2ms/step - loss: 0.6458 - val_loss: 0.6689
Epoch 4/10
1777/1777 [==============================] - 6s 3ms/step - loss: 0.6450 - val_loss: 0.6682
Epoch 5/10
1777/1777 [==============================] - 4s 2ms/step - loss: 0.6448 - val_loss: 0.6680
Epoch 6/10
1777/1777 [==============================] - 4s 2ms/step - loss: 0.6446 - val_loss: 0.6679
Epoch 7/10
1777/1777 [==============================] - 6s 3ms/step - loss: 0.6446 - val_loss: 0.6680
Epoch 8/10
1777/1777 [==============================] - 4s 2ms/step - loss: 0.6414 - val_loss: 0.6617
Epoch 9/10
1777/1777 [==============================] - 4s 2ms/step - loss: 0.6377 - val_loss: 0.6602
Epoch 10/10
1781/1781 [==============================] - 2s 1ms/step
[[56419   445